In [0]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, ZeroPadding2D
from keras.layers import BatchNormalization, Activation, Dropout
from keras import regularizers
import matplotlib.pyplot as plt

In [0]:
(x_train, _), (x_test, _) = mnist.load_data()

# Scales the training and test data to range between 0 and 1.
max_value = float(x_train.max())
x_train = x_train.astype('float32') / max_value
x_test = x_test.astype('float32') / max_value
x_train = x_train.reshape((len(x_train), 28, 28, 1))
x_test = x_test.reshape((len(x_test), 28, 28, 1))

In [14]:
autoencoder = Sequential()

autoencoder.add(Conv2D(96, (11, 11), input_shape=x_train.shape[1:], padding='same'))
autoencoder.add(BatchNormalization())
autoencoder.add(Activation('relu'))
autoencoder.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
autoencoder.add(Conv2D(256, (5, 5), padding='same'))
autoencoder.add(BatchNormalization())
autoencoder.add(Activation('relu'))
autoencoder.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3
autoencoder.add(ZeroPadding2D((1, 1)))
autoencoder.add(Conv2D(512, (3, 3), padding='same'))
autoencoder.add(BatchNormalization())
autoencoder.add(Activation('relu'))
autoencoder.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4
autoencoder.add(ZeroPadding2D((1, 1)))
autoencoder.add(Conv2D(1024, (3, 3), padding='same'))
autoencoder.add(BatchNormalization())
autoencoder.add(Activation('relu'))

# Layer 5
autoencoder.add(ZeroPadding2D((1, 1)))
autoencoder.add(Conv2D(1024, (3, 3), padding='same'))
autoencoder.add(BatchNormalization())
autoencoder.add(Activation('relu'))
autoencoder.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 6
autoencoder.add(Flatten())
autoencoder.add(Dense(3072))
autoencoder.add(BatchNormalization())
autoencoder.add(Activation('relu'))
autoencoder.add(Dropout(0.5))

# Layer 7
autoencoder.add(Dense(4096))
autoencoder.add(BatchNormalization())
autoencoder.add(Activation('relu'))
autoencoder.add(Dropout(0.5))

## flatten for output
autoencoder.add(Dense(128))
autoencoder.add(Reshape((4,4,8)))

## decode latent code
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(16, (3, 3), activation='relu'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 28, 28, 96)        11712     
_________________________________________________________________
batch_normalization_31 (Batc (None, 28, 28, 96)        384       
_________________________________________________________________
activation_31 (Activation)   (None, 28, 28, 96)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 14, 14, 256)       614656    
_________________________________________________________________
batch_normalization_32 (Batc (None, 14, 14, 256)       1024      
_________________________________________________________________
activation_32 (Activation)   (None, 14, 14, 256)       0         
__________

In [15]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dense_14').output)
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24_input (InputLayer) (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 28, 28, 96)        11712     
_________________________________________________________________
batch_normalization_31 (Batc (None, 28, 28, 96)        384       
_________________________________________________________________
activation_31 (Activation)   (None, 28, 28, 96)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 14, 14, 256)       614656    
_________________________________________________________________
batch_normalization_32 (Batc (None, 14, 14, 256)       1024      
__________

In [0]:
## Because I used the AlexNet architecture here, the training process is left as an exercise.

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=128,
                validation_data=(x_test, x_test))